In [1]:
import tensorflow as tf

In [2]:
import pickle

In [3]:
model=tf.keras.models.load_model("../models/pos_model.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 300)          15969600  
                                                                 
 lstm (LSTM)                 (None, 100, 256)          570368    
                                                                 
 time_distributed (TimeDistr  (None, 100, 13)          3341      
 ibuted)                                                         
                                                                 
Total params: 16,543,309
Trainable params: 16,543,309
Non-trainable params: 0
_________________________________________________________________


In [4]:
words_dict=pickle.load(open("../binary_information/word.pickle","rb"))
len(words_dict)

53232

In [5]:
words_dict["he"]

13

In [7]:
words_dict["love"]

463

In [9]:
words_dict["studying"]

2888

In [10]:
idx_2_word={idx:word for word,idx in words_dict.items()}
idx_2_word[2]

','

In [11]:
idx_2_word[13]

'he'

In [12]:
classes=pickle.load(open("../binary_information/classes.pickle","rb"))
classes

{'noun': 1,
 'verb': 2,
 '.': 3,
 'adp': 4,
 'det': 5,
 'adj': 6,
 'adv': 7,
 'pron': 8,
 'conj': 9,
 'prt': 10,
 'num': 11,
 'x': 12}

In [13]:
idx_2_class={idx:word for word,idx in classes.items()}

In [16]:
def process_data(text):
    arr=text.split(" ")
    arr=[words_dict[w] for w in arr]
    padded=tf.keras.preprocessing.sequence.pad_sequences([arr],maxlen=100)
    return padded

In [17]:
process_data("he love studying")

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   13,  463,
        2888]])

In [25]:
pred=model.predict(process_data("he love studying"))

1/1 [==============================] - 0s 34ms/step


In [26]:
import numpy as np
pred=[np.argmax(num) for num in pred[0]]

In [28]:
pred2=[num for num in pred if num != 0]
pred2

[8, 2, 2]

In [29]:
[idx_2_class[num] for num in pred2]

['pron', 'verb', 'verb']

In [31]:
def part_of_speech(text):
    words=text.split(" ")
    padding=process_data(text)
    pred=model.predict(padding)
    pred=[np.argmax(num) for num in pred[0]]
    pred=[num for num in pred if num != 0]
    pos_dict={words[num]:idx_2_class[pred[num]] for num in range(len(pred))}
    return pos_dict
part_of_speech("he love playing")

1/1 [==============================] - 0s 33ms/step


{'he': 'pron', 'love': 'verb', 'playing': 'verb'}

In [32]:
part_of_speech("the place which i visit it is very boring")

1/1 [==============================] - 0s 37ms/step


{'the': 'det',
 'place': 'noun',
 'which': 'det',
 'i': 'pron',
 'visit': 'verb',
 'it': 'pron',
 'is': 'verb',
 'very': 'adv',
 'boring': 'verb'}

In [33]:
part_of_speech("john play football")

1/1 [==============================] - 0s 55ms/step


{'john': 'noun', 'play': 'verb', 'football': 'noun'}